In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import dgl

cstr = pd.read_csv('data/dataset/cstr_data.csv')
feedin = pd.read_csv('data/dataset/feedin_data.csv')
feedout = pd.read_csv('data/dataset/feedout_data.csv')
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def generate_words(row):
    fields = [
        'state', #
        'vapor_cloud_formation',#
        'combustible',#
        'confinement',#
        'mechanical_failure',#
        'release',#
        'overflow',#
        'vaporization',#
        'temperature',#
        'pressure',#
        'ignition',#
        'pre_mixture',
        'toxicity_of_sub',
        'dispersion'
    ]
    words = [f"{field} {row[field]}" for field in fields]
    return words

cstr_word = cstr.apply(generate_words, axis=1)
feedin_word = feedin.apply(generate_words, axis=1)
feedout_word = feedout.apply(generate_words, axis=1)

In [ ]:
# 将列表形式的 Series 拆分为 DataFrame
cstr_word_df = pd.DataFrame(cstr_word.tolist(), columns=[
    'state',
    'vapor_cloud_formation',
    'combustible',
    'confinement',
    'mechanical_failure',
    'release',
    'overflow',
    'vaporization',
    'temperature',
    'pressure',
    'ignition',
    'pre_mixture',
    'toxicity_of_sub',
    'dispersion'
])
feedin_word_df = pd.DataFrame(feedin_word.tolist(), columns=[
    'state',
    'vapor_cloud_formation',
    'combustible',
    'confinement',
    'mechanical_failure',
    'release',
    'overflow',
    'vaporization',
    'temperature',
    'pressure',
    'ignition',
    'pre_mixture',
    'toxicity_of_sub',
    'dispersion'
])

feedout_word_df = pd.DataFrame(feedout_word.tolist(), columns=[
    'state',
    'vapor_cloud_formation',
    'combustible',
    'confinement',
    'mechanical_failure',
    'release',
    'overflow',
    'vaporization',
    'temperature',
    'pressure',
    'ignition',
    'pre_mixture',
    'toxicity_of_sub',
    'dispersion'
])
feedin_word_df

,state,vapor_cloud_formation,combustible,confinement,mechanical_failure,release,overflow,vaporization,temperature,pressure,ignition,pre_mixture,toxicity_of_sub,dispersion
0,state vapor,vapor_cloud_formation False,combustible False,confinement high,mechanical_failure False,release False,overflow False,vaporization none,temperature low,pressure high,ignition False,pre_mixture False,toxicity_of_sub False,dispersion False
1,state vapor,vapor_cloud_formation False,combustible False,confinement high,mechanical_failure False,release False,overflow False,vaporization none,temperature low,pressure low,ignition False,pre_mixture False,toxicity_of_sub False,dispersion False
2,state vapor,vapor_cloud_formation False,combustible False,confinement high,mechanical_failure False,release False,overflow False,vaporization none,temperature low,pressure high,ignition False,pre_mixture False,toxicity_of_sub False,dispersion False
3,state vapor,vapor_cloud_formation False,combustible False,confinement high,mechanical_failure False,release False,overflow False,vaporization none,temperature low,pressure high,ignition False,pre_mixture False,toxicity_of_sub False,dispersion False
4,state vapor,vapor_cloud_formation False,combustible False,confinement high,mechanical_failure False,release False,overflow False,vaporization none,temperature low,pressure high,ignition False,pre_mixture False,toxicity_of_sub False,dispersion False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,state vapor,vapor_cloud_formation False,combustible False,confinement low,mechanical_failure False,release False,overflow False,vaporization high,temperature low,pressure high,ignition False,pre_mixture False,toxicity_of_sub False,dispersion False
7596,state solid,vapor_cloud_formation False,combustible False,confinement high,mechanical_failure False,release False,overflow False,vaporization high,temperature low,pressure low,ignition False,pre_mixture False,toxicity_of_sub False,dispersion False
7597,state vapor,vapor_cloud_formation False,combustible False,confinement low,mechanical_failure False,release False,overflow False,vaporization low,temperature low,pressure high,ignition False,pre_mixture False,toxicity_of_sub True,dispersion False
7598,state liquid,vapor_cloud_formation False,combustible False,confinement low,mechanical_failure False,release False,overflow False,vaporization high,temperature low,pressure high,ignition False,pre_mixture False,toxicity_of_sub True,dispersion False


In [4]:
cstr_word_df['combined_sentence'] = cstr_word_df.apply(lambda row: ','.join(row.astype(str)), axis=1)
feedin_word_df['combined_sentence'] = feedin_word_df.apply(lambda row: ','.join(row.astype(str)), axis=1)
feedout_word_df['combined_sentence'] = feedout_word_df.apply(lambda row: ','.join(row.astype(str)), axis=1)

In [5]:
cstr_word_df['label'] = None
cstr_word_df['scenario'] = None
cstr_word_df['label'] = cstr['label']

In [ ]:
import json
# load json file
json_file = "data/dataset/graph_relations_labeled_structured.json"
with open(json_file, "r") as file:
    scenarios_data = json.load(file)

for i in range(len(cstr_word_df)):
    label = cstr_word_df["label"].iloc[i] 

    for scenario in scenarios_data:  # 
        if label.lower() == scenario["Hazard"].lower():
            
            pretty_text = " <newline> ".join(scenario["Scenarios"])
            cstr_word_df.at[i, "scenario"] = pretty_text
            break
cstr_word_df

,state,vapor_cloud_formation,combustible,confinement,mechanical_failure,release,overflow,vaporization,temperature,pressure,ignition,pre_mixture,toxicity_of_sub,dispersion,combined_sentence,label,scenario
0,state vapor,vapor_cloud_formation True,combustible True,confinement high,mechanical_failure True,release instant,overflow False,vaporization none,temperature high,pressure high,ignition delayed,pre_mixture True,toxicity_of_sub False,dispersion False,"state vapor,vapor_cloud_formation True,combust...",VCE_vapor_instant,Pre-mixture causes Vapor cloud formation <newl...
1,state vapor,vapor_cloud_formation True,combustible True,confinement high,mechanical_failure True,release instant,overflow False,vaporization none,temperature low,pressure low,ignition delayed,pre_mixture True,toxicity_of_sub False,dispersion False,"state vapor,vapor_cloud_formation True,combust...",VCE_vapor_instant,Pre-mixture causes Vapor cloud formation <newl...
2,state vapor,vapor_cloud_formation True,combustible True,confinement high,mechanical_failure True,release instant,overflow False,vaporization none,temperature high,pressure high,ignition delayed,pre_mixture True,toxicity_of_sub False,dispersion True,"state vapor,vapor_cloud_formation True,combust...",VCE_vapor_instant,Pre-mixture causes Vapor cloud formation <newl...
3,state vapor,vapor_cloud_formation True,combustible True,confinement high,mechanical_failure True,release instant,overflow False,vaporization none,temperature high,pressure high,ignition delayed,pre_mixture True,toxicity_of_sub False,dispersion True,"state vapor,vapor_cloud_formation True,combust...",VCE_vapor_instant,Pre-mixture causes Vapor cloud formation <newl...
4,state vapor,vapor_cloud_formation True,combustible True,confinement high,mechanical_failure True,release instant,overflow False,vaporization none,temperature low,pressure high,ignition delayed,pre_mixture True,toxicity_of_sub False,dispersion False,"state vapor,vapor_cloud_formation True,combust...",VCE_vapor_instant,Pre-mixture causes Vapor cloud formation <newl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,state vapor,vapor_cloud_formation False,combustible True,confinement low,mechanical_failure False,release instant,overflow True,vaporization high,temperature high,pressure high,ignition false,pre_mixture True,toxicity_of_sub False,dispersion True,"state vapor,vapor_cloud_formation False,combus...",Normal,Normal causes Normal
7596,state solid,vapor_cloud_formation False,combustible True,confinement high,mechanical_failure False,release continuous,overflow False,vaporization high,temperature low,pressure low,ignition true,pre_mixture True,toxicity_of_sub False,dispersion True,"state solid,vapor_cloud_formation False,combus...",Normal,Normal causes Normal
7597,state vapor,vapor_cloud_formation True,combustible False,confinement low,mechanical_failure True,release continuous,overflow True,vaporization low,temperature low,pressure high,ignition true,pre_mixture True,toxicity_of_sub True,dispersion True,"state vapor,vapor_cloud_formation True,combust...",Normal,Normal causes Normal
7598,state liquid,vapor_cloud_formation True,combustible False,confinement low,mechanical_failure False,release continuous,overflow False,vaporization high,temperature high,pressure high,ignition true,pre_mixture False,toxicity_of_sub True,dispersion False,"state liquid,vapor_cloud_formation True,combus...",Normal,Normal causes Normal


In [7]:
cstr_word_df.to_csv("data//dataset/cstr_combined.csv", index=False, encoding="utf-8")
feedin_word_df.to_csv("data//dataset/feedin_combined.csv", index=False, encoding="utf-8")
feedout_word_df.to_csv("data//dataset/feedout_combined.csv", index=False, encoding="utf-8")

In [8]:
label_counts = cstr_word_df["label"].value_counts()
print(label_counts)

label
Normal                                3843
BLEVE                                  602
toxicity_liquid_overflow               228
toxicity_liquid_mechanical_failure     224
VCE_vapor_continuous                   200
toxicity_vapor_overflow                200
toxicity_vapor_mechanical_failure      200
flash_fire_vapor                       200
flash_fire_liquid                      200
VCE_vapor_instant                      200
jet_fire_liquid                        200
Pool_fire_mechanical_failure           200
Pool_fire_overflow                     200
dust_explosion                         200
VCE_liquid_continuous                  200
VCE_liquid_instant                     200
jet_fire_vapor                         200
fire_ball                              103
Name: count, dtype: int64


In [ ]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
from tqdm import tqdm
import pandas as pd

tokenizer = T5Tokenizer.from_pretrained('t5-small')
encoder = T5EncoderModel.from_pretrained('t5-small')
encoder.eval()  # 
def encode_series_to_embeddings(series_of_words, encoder, tokenizer, max_length=10, device=device):
    input_ids_all = []
    attention_masks_all = []

    print("Tokenizing...")

    for word_list in tqdm(series_of_words):
        encoding = tokenizer(
            word_list,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='pt'
        )
        input_ids_all.append(encoding['input_ids'])        # [12, max_length]
        attention_masks_all.append(encoding['attention_mask'])

    input_ids = torch.stack(input_ids_all).to(device)              # [1600, 12, max_length]
    attention_mask = torch.stack(attention_masks_all).to(device)   # [1600, 12, max_length]

    batch_size, num_nodes, seq_len = input_ids.shape

    input_ids_flat = input_ids.view(-1, seq_len)             # [1600*12, max_length]
    attention_mask_flat = attention_mask.view(-1, seq_len)

    print("Encoding with T5...")

    with torch.no_grad():
        encoder = encoder.to(device)
        outputs = encoder(input_ids=input_ids_flat, attention_mask=attention_mask_flat)
        last_hidden_state = outputs.last_hidden_state  

    node_embeddings = last_hidden_state[:, 0, :]  

    # reshape to [batch_size, 11, 512]
    node_embeddings = node_embeddings.view(batch_size, num_nodes, -1)

    return node_embeddings  # shape: [1600, 12, 512]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
import torch
from transformers import T5Tokenizer, T5EncoderModel
from tqdm import tqdm

def encode_series_to_embeddings(series_of_words, encoder, tokenizer, max_length, batch_size, device="cuda",):
    encoder = encoder.to(device)
    encoder.eval()

    all_embeddings = []

    print("Encoding in batches...")

    for i in tqdm(range(0, len(series_of_words), batch_size)):
        batch_graphs = series_of_words[i : i + batch_size]  

        input_ids_all = []
        attention_masks_all = []

        for word_list in batch_graphs:
            encoding = tokenizer(
                word_list,
                padding='max_length',
                truncation=True,
                max_length=max_length,
                return_tensors='pt'
            )
            input_ids_all.append(encoding['input_ids'])           # [15, max_length]
            attention_masks_all.append(encoding['attention_mask'])

        input_ids = torch.stack(input_ids_all).to(device)         # [B, 15, max_length]
        attention_mask = torch.stack(attention_masks_all).to(device)

        B, num_nodes, seq_len = input_ids.shape
        input_ids_flat = input_ids.view(-1, seq_len)              # [B*15, max_length]
        attention_mask_flat = attention_mask.view(-1, seq_len)

        with torch.no_grad():
            outputs = encoder(input_ids=input_ids_flat, attention_mask=attention_mask_flat)
            last_hidden_state = outputs.last_hidden_state         # [B*15, max_length, 512]

        node_embeddings = last_hidden_state[:, 0, :]              #  [CLS] token → [B*15, 512]
        node_embeddings = node_embeddings.view(B, num_nodes, -1)  # reshape to [B, 15, 512]

        all_embeddings.append(node_embeddings.cpu())

    return torch.cat(all_embeddings, dim=0)  # [N, 15, 512]


In [ ]:
cstr_word_df['combined_sentence'][0]

token_lengths = [len(tokenizer.tokenize(text)) for text in cstr_word_df['combined_sentence']]
print(f"最大长度: {max(token_lengths)}")
max_token_lengths = max(token_lengths)


最大长度: 83


In [ ]:
cstr_sentence = cstr_word_df['combined_sentence'].tolist()  # List[str]
cstr_graph_texts = [[text] for text in cstr_sentence]            # List[List[str]]
feedin_sentence = feedin_word_df['combined_sentence'].tolist()  # List[str]
feedin_graph_texts = [[text] for text in feedin_sentence]            # List[List[str]]
feedout_sentence = feedin_word_df['combined_sentence'].tolist()  # List[str]
feedout_graph_texts = [[text] for text in feedout_sentence]            # List[List[str]]

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
encoder = T5EncoderModel.from_pretrained('t5-small')

cstr_embeddings = encode_series_to_embeddings(cstr_graph_texts, encoder, tokenizer, max_length=max_token_lengths, device='cuda', batch_size=64)
feedin_embeddings = encode_series_to_embeddings(feedin_graph_texts, encoder, tokenizer, max_length=max_token_lengths, device='cuda', batch_size=64)
feedout_embeddings = encode_series_to_embeddings(feedout_graph_texts, encoder, tokenizer, max_length=max_token_lengths, device='cuda', batch_size=64)
print(cstr_embeddings.shape) 


Encoding in batches...


100%|██████████| 119/119 [00:07<00:00, 16.45it/s]


Encoding in batches...


100%|██████████| 119/119 [00:07<00:00, 15.02it/s]


Encoding in batches...


100%|██████████| 119/119 [00:07<00:00, 16.85it/s]

torch.Size([7600, 1, 512])


In [14]:
combined = torch.cat([feedin_embeddings, cstr_embeddings, feedout_embeddings], dim=1)
print(combined.shape)  # [7600, 1+X, 512]

torch.Size([7600, 3, 512])


In [15]:
import dgl
import torch

def create_dgl_graph_from_embedding(embeddings: torch.Tensor):
    """
    Create a list of DGLGraphs from embeddings.
    Each graph has chain edges: node 0 -> node 1 -> ... -> node (n-1)

    The graphs will be created on the same device as embeddings.
    
    :param embeddings: Tensor of shape [batch_size, num_nodes, feature_dim]
    :return: List of DGLGraphs
    """
    batch_size, num_nodes, feature_dim = embeddings.shape
    device = embeddings.device

    graph_list = []

    for i in range(batch_size):
        g = dgl.DGLGraph().to(device)
        g.add_nodes(num_nodes)

        # Create chain edges: 0→1, 1→2, ..., (n-2)→(n-1)
        src_nodes = torch.arange(0, num_nodes - 1, device=device)
        dst_nodes = torch.arange(1, num_nodes, device=device)
        g.add_edges(src_nodes, dst_nodes)

        # Assign node features
        g.ndata['h'] = embeddings[i]

        graph_list.append(g)

    return graph_list


In [16]:
graph_list = create_dgl_graph_from_embedding(combined)

print(graph_list[0])
print(graph_list[0].ndata['h'].shape) 
dgl.save_graphs("data/dataset/graph_data.bin", graph_list)


/home/chenyangtamu/miniconda3/envs/graphdecoder310/lib/python3.10/site-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


Graph(num_nodes=3, num_edges=2,
      ndata_schemes={'h': Scheme(shape=(512,), dtype=torch.float32)}
      edata_schemes={})
torch.Size([3, 512])
